<a href="https://colab.research.google.com/github/SveinungOverland/ZeroGO/blob/master/NN/DCNN_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Convolutional Neural Net for ZeroGO V1

In [108]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

tf.__version__

'2.0.0'

## Layers and heads

### Residual layer

In [0]:
def add_residual_layer(x):
  res = layers.Conv2D(256, kernel_size=(3,3), padding='same')(x)
  res = layers.BatchNormalization(axis=1)(res)
  res = layers.ReLU()(res)
  res = layers.Conv2D(256, kernel_size=(3,3), padding='same')(res)
  res = layers.BatchNormalization(axis=1)(res)
  res = layers.ReLU()(res)
  res = layers.add([res, x])
  return layers.ReLU()(res)

### Create trunk

In [0]:
def create_trunk(shape, nr_residual_layers, filters):
  inputs = keras.Input(shape=shape)
  x = layers.Conv2D(filters, kernel_size=(3,3), padding='same')(inputs)
  x = layers.BatchNormalization(axis=1)(x)
  x = layers.ReLU()(x)
  for i in range(nr_residual_layers):
    x = add_residual_layer(x)
  return keras.Model(inputs, x)

### Create value head

In [0]:
def create_value_head(shape, filters):
    return keras.Sequential([
        layers.Conv2D(1, kernel_size=(1,1), padding='same'),
        layers.BatchNormalization(axis=1),
        layers.ReLU(),
        layers.Dense(filters),
        layers.ReLU(),
        layers.Dense(1, activation="tanh")
    ])

### Create policy head

In [0]:
def create_policy_head(shape, filters):
  return keras.Sequential([
      layers.Conv2D(2, kernel_size=(1,1), padding='same'),
      layers.BatchNormalization(axis=1),
      layers.ReLU(),
      layers.Dense(shape[0] * shape[1] + 1)
  ])

## Mode class

In [0]:
class Mode:
  Trunk = 0
  Value = 1
  Policy = 2
  ValueHead = 3
  PolicyHead = 4

## Model class 

In [0]:
class Model:
  def __init__(self, trunk, value_head, policy_head):    
    # Can be choosen by mode
    self.trunk = trunk # Is a model
    self.value_head = value_head
    self.policy_head = policy_head
  
    # Can also be choosen by mode
    self.value_path = keras.Sequential([self.trunk, self.value_head])
    self.policy_path = keras.Sequential([self.trunk, self.policy_head])
    
  def save(self, file_name: str):
    print("Can't save yet")
    pass
  
#   def loss_fn(self, value_true, value_pred, policy_true, policy_pred):
#     # (z - v)^2 - pi^T * log(p) + c*||O||^2, der z = self-play-winner, v = predicted value, pi = search probabilities, p = nn move probability, O = nn parameters (weights), c = parameter controlling the level of L2 weight regularisation to prevent overfitting
#     # sum of mean-squared error and cross-entropy loss
#     return keras.losses.MSE(y_true, y_pred) + keras.losses.categorical_crossentropy(policy_true, policy_pred) # v1
#     pass
  
  def get_trunk_weights(self):
    return self.trunk.get_weights()
  
  def __retrieve_net(self, mode):
    if mode == Mode.Trunk:
      net = self.trunk
    elif mode == Mode.Value:
      net = self.value_path
    elif mode == Mode.Policy:
      net = self.policy_path
    elif mode == Mode.ValueHead:
      net = self.value_head
    elif mode == Mode.PolicyHead:
      net = self.policy_head
    else:
      print("No acceptible mode given!")
      raise Exception("No acceptible mode given!")
    return net
  
  def train(self, mode: Mode, loss):
    net = self.__retrieve_net(mode)
    tape = tf.GradientTape()
    gradients = tape.gradient(loss, net.trainable_variables)
    optimizer.apply_gradients(zip(gradients, net.trainable_variables))
  
  def predict(self, mode, X):
    net = self.__retrieve_net(mode)
    return net.predict(X)
  
  def describe(self, mode):
    net = self.__retrieve_net(mode)
    return net.summary()
  
  @classmethod
  def load(cls, filename):
    print("Can't load yet")
    pass
  
  @classmethod
  def create(cls, shape=(5,5,7), nr_residual_layers=10, filters=256):
    trunk = create_trunk(shape, nr_residual_layers, filters)
    value_head = create_value_head(shape, filters)
    policy_head = create_policy_head(shape, filters)
    return cls(trunk, value_head, policy_head)

## Test

In [116]:
model = Model.create(shape=(2,2,1), nr_residual_layers=2)

state = np.array([
    [[0], [1]],
    [[0], [0]]
])
print("Shape: ", state.shape)

print("Value summary: ", model.describe(Mode.Value))
print()
print("Policy summary: ", model.describe(Mode.Policy))
print()
print("Value: ", model.predict(Mode.Value, state))
print("Policy: ", model.predict(Mode.Policy, state))

Shape:  (2, 2, 1)
Model: "sequential_66"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_12 (Model)             (None, 2, 2, 256)         2362920   
_________________________________________________________________
sequential_64 (Sequential)   (None, 2, 2, 1)           1034      
Total params: 2,363,954
Trainable params: 2,363,930
Non-trainable params: 24
_________________________________________________________________
Value summary:  None

Model: "sequential_67"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_12 (Model)             (None, 2, 2, 256)         2362920   
_________________________________________________________________
sequential_65 (Sequential)   (None, 2, 2, 5)           537       
Total params: 2,363,457
Trainable params: 2,363,433
Non-trainable params: 24
__________________________________

ValueError: ignored